In [1]:
import os
import collections
import numpy as np
from keras.preprocessing.sequence import skipgrams
from keras.preprocessing import sequence
import pickle as pickle
import re

Using TensorFlow backend.
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\yoshi\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
def read_data(filename):
    try:
        file=open(os.path.join(os.path.join(os.path.abspath('../binaries'),"data"),filename),"r", encoding="utf-8")
        return re.sub(r"[^\w\s]", ' ',file.read().lower()).split()
    except:
        raise Exception("../binaries/data/"+filename+" does not exist.")

In [3]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
        data.append(index)
    return data, dictionary

In [4]:
def collect_data(vocabulary_size=25000):
    vocabulary = read_data("data.txt")
    data,dictionary = build_dataset(vocabulary,vocabulary_size)
    del vocabulary
    return data, dictionary 

In [5]:
vocab_size = 100000
data, dictionary = collect_data(vocabulary_size=vocab_size)
window_size = 3

In [6]:
sampling_table = sequence.make_sampling_table(vocab_size)
couples, labels = skipgrams(data, vocab_size, window_size=window_size, sampling_table=sampling_table)
word_target, word_context = zip(*couples)
word_target = np.array(word_target, dtype="int32")
word_context = np.array(word_context, dtype="int32")

In [7]:
names=["window_size","vocab_size","dictionary","labels","word_target","word_context"]
for name in names:
    with open(os.path.join(os.path.join(os.path.abspath('../binaries'),"pickles"),name+".pickle"), "wb") as f:
        pickle.dump(globals()[name], f)